## 使用davinci-003推理

In [1]:
import os


In [2]:

import logging
from collections import Counter

from utils import  load_data,getKey,write_json,mkpath
 
Result_Folder = "jx_result"
Err_Result_Folder = "jx_err_result"
dataset = "AddSub"
datapath = "./dataset/AddSub/AddSub_200_1.json"

In [3]:
import datetime

# 获取当前时间
current_time = datetime.datetime.now()

# 将时间格式化为文件夹名称的形式
format_time = current_time.strftime("%m%d_%H%M")

print("时间:", format_time)

if not os.path.exists(Result_Folder):
    mkpath(Result_Folder)
if not os.path.exists(f"{Result_Folder}/{dataset}"):
    mkpath(f'{Result_Folder}/{dataset}')
if not os.path.exists(Err_Result_Folder):
    mkpath(Err_Result_Folder)
if not os.path.exists(f"{Err_Result_Folder}/{dataset}"):
    mkpath(f'{Err_Result_Folder}/{dataset}')


时间: 0104_1037


In [4]:
question, answer, ids = load_data(datapath,dataset)
assert len(question) == len(answer), "输入长度不一致"
assert len(question) == len(ids),"输入长度不一致"
data_scale = len(answer)
print(f"问题个数：{len(question)}")
print(f"答案个数：{len(answer)}")

question[0],answer[0]

问题个数：195
答案个数：195


("At the beach , Miki and her sister both built sandcastles and then measured their heights . Miki 's sandcastle was 0.8333333333333334 foot tall and her sister 's was 0.5 foot tall . How much taller was Miki 's sandcastle than her sister 's ? ",
 0.3333333333333333)

## 单条prompt

### 两个list分别代表201和202的所有问题的cots;每个列表是n个字典，字典内包含cots

In [5]:
from prepare_prompt import get_prompt,construct_input
from prediction_runner import basic_runner
from extracter import extract_answer,get_precision
from tqdm import tqdm


engine = "text-davinci-003"
max_length_cot = 512
key_index = 0


def get_paths(prompt_id,prompt_type,path_num):
    res = []
    _, prompt = get_prompt(prompt_id=prompt_id,type=prompt_type)
    # print(prompt)
    Predict_File = f'{Result_Folder}/{dataset}/{format_time}-Type_{prompt_type}-{prompt_id}-Num_{data_scale}-{path_num}.json'
    Decoder_Error_File = f'{Err_Result_Folder}/{dataset}/{format_time}-Type_{prompt_type}-{prompt_id}-{path_num}.json'
    pred_list = []
    for idx, element in tqdm(enumerate(question)):
        inputs = construct_input(prompt, element)
        try:
            get_result, pred_list, error_msg = basic_runner(engine, inputs, max_length_cot, key_index,path_num)
            res.append({element:pred_list})
        except Exception as e:
            print("没有获取到答案！\n")
            decode_error_data = {
                'question': question[idx],
                'answer':answer[idx]
            }
            write_json(decode_error_data, Decoder_Error_File)
            print(
                f"an error raised when predicting (question id: {ids[idx]}). "
                f"ERROR: {getattr(e.__class__, '__name__')}:{str(e)}"
            )
            continue   
        # print("pred_list:",pred_list)        
        if not get_result:
            print(
                f"not get predicted result (question id: {ids[idx]})."
                f"ERROR Message: {error_msg if error_msg else None}"
            )
            continue
        json_data = {
                        "ID": ids[idx],
                        "question": question[idx],
                        "chain-of-thought": pred_list,
                        # "pred": pred_answer,
                        "answer": answer[idx],
                        # "ans": ans
                    }    
        write_json(json_data, Predict_File)

    return res

In [6]:


def get_all_paths(prompt_id_list,path_num,prompt_type):
    res = []
    for id in prompt_id_list:
        paths = get_paths(prompt_id=id,prompt_type=prompt_type,path_num=path_num)
        res.append(paths)
    return res

def get_combined_paths(res_list:list):
    result = []
    assert len(res_list) == 2,"prompt限制两条"
    paths_1 = res_list[0]
    paths_2 = res_list[1]
    # print(len(paths_1))
    assert len(paths_1) == len(question) == len(paths_2),"path个数和问题个数不匹配"

    for j in range(len(paths_1)):
        for k,v in paths_1[j].items():
            k1,v1 = k,v
        for k,v in paths_2[j].items():
            k2,v2 = k,v
        # print('----------------')
        # print(k1)
        # print(k2)
        # print(question[j])
        # print('----------------')
        assert k1 == k2 == question[j]
        temp = {
            "question" : k1,
            "paths" : v1 + v2,
            "answer" : answer[j]
        }
            
        result.append(temp)
    return result

# test_a = [[{'a':[1]},{'b':[2]}],[{'a':[3]},{'b':[4]}]]
# get_combined_paths(test_a)
        

In [16]:
# import json 
# f1 = []
# f2 = []
# with open('./jx_result/gsm8k/1101_1812-Type_1-201-Num_266-5.json','r',encoding='utf-8') as f11:
#     f1 = json.load(f11)
# with open('./jx_result/gsm8k/1101_1909-Type_1-202-Num_266-5.json','r',encoding='utf-8') as f22:
#     f2 = json.load(f22)    
# len(f1),len(f2)

In [17]:
# t1 = []
# t2 = []
# for i in range(len(f1)):
#     temp1 = {}
#     temp2 = {}
#     q1 = f1[i]['question']
#     v1 = f1[i]['chain-of-thought']
#     temp1[q1] = v1
#     t1.append(temp1)
#     q2 = f2[i]['question']
#     v2 = f2[i]['chain-of-thought']
#     temp2[q2] = v2
#     t2.append(temp2)
# all_paths = []
# all_paths.append(t1)
# all_paths.append(t2)

In [8]:
91.8+59.3+92.2+46+94.7+75.7

459.7

In [12]:
(46+59.3+91.8+92.2+75.7)/5

73.0

In [14]:
(44.5+60.7+93.3+92.9+78.5)/5

73.97999999999999

In [7]:
prompt_id_list = [211,212]
path_num = 5
prompt_type = 1
""
 

all_paths = get_all_paths(prompt_id_list=prompt_id_list,path_num=path_num,prompt_type=prompt_type)
# res_paths = get_combined_paths(all_paths)

0it [00:00, ?it/s]

104it [10:21,  5.99s/it]

达到每日200条限制的key： sess-VpQIrNTw4RryCWl7zBDV6yGaJ7s7sOemhpvy5uKB


107it [10:46,  7.56s/it]

达到每日200条限制的key： sess-VpQIrNTw4RryCWl7zBDV6yGaJ7s7sOemhpvy5uKB


...

190it [28:47,  6.89s/it]

达到每日200条限制的key： sess-m1vvBQ2trxJuuKZfYGB3NPybVBeaMzBW1Rg9EPYw


193it [29:10,  7.25s/it]

达到每日200条限制的key： sess-m1vvBQ2trxJuuKZfYGB3NPybVBeaMzBW1Rg9EPYw


195it [29:23,  9.04s/it]


In [ ]:
assert len(prompt_id_list) == 2
ids = prompt_id_list[0] +prompt_id_list[1]

temp_p = f'{Result_Folder}/{dataset}/temp-{format_time}-Type_{prompt_type}-{ids}-Num_{data_scale}-{path_num}.json'
# temp_p = './jx_result/gsm8k/temp-1101_1812-Type_1-201-Num_266-5.json'
write_json(res_paths,path=temp_p)

## 使用chatGLM抽取答案

In [1]:
import re

text = "\nPlan a Solution: \n1. Identify the words in the given string. \n2. Take the last letter of each word. \n3. Concatenate the last letters. \n\nSolve the Problem: \n1. Identify the words in the given string: \"Lucky\", \"Mireya\", \"Jj\", \"Kc\". \n2. Take the last letter of each word: \"y\", \"a\", \"j\", \"c\". \n3. Concatenate the last letters: \"yajc\"."

re.sub("\"|\'|\n|\.|\s", "", text)

'PlanaSolution:1Identifythewordsinthegivenstring2Takethelastletterofeachword3ConcatenatethelastlettersSolvetheProblem:1Identifythewordsinthegivenstring:Lucky,Mireya,Jj,Kc2Takethelastletterofeachword:y,a,j,c3Concatenatethelastletters:yajc'

In [4]:
text = "The answer is xaqud".split('The answer is')[-1]

re.sub("\"|\'|\n|\.|\s", "", text)

'xaqud'

### 准备数据

In [ ]:
import json

# temp_p = "./jx_result/gsm8k/temp-1031_0933-Type_1-403-Num_100-5.json"
with open(temp_p,'r') as f:
    data = json.load(f)
len(data),data[0]

### 抽取答案

In [ ]:
def chatGLM(text,history):
    res,history  = model.chat(tokenizer,text,history)
#     print(res)
    return res

In [ ]:




induce = "Please extract the answer from the text.\n[Output]: The answer is xxx.\n"

for item in data:
    cots = item['paths']
    ans = []
    for cot in cots:
        induce_prompt = induce + '[Text]:\n' + cot
        try:
            res = chatGLM(induce_prompt,[])
            ans.append(res)
        except Exception as e:
            print(e)
    item['answers'] = ans    
   

In [ ]:
import re

def extract_number(text:str):
    pattern = r'(?<=answer\s).*?(\d+(?:\.\d+)?)'
    match = re.search(pattern=pattern,string=text)
    if match:
        answer = float(match.group(1))
    else:
        answer = None
    return answer


def extract_answer(text: str):
    
    pattern_1 = r'\d+(?:\.\d+)?'
    # 尝试直接从cot中抽取答案
    answer = extract_number(text)
    # 如果抽不出来，则将其再次输入chatGLM-6b,得到新的text
    if answer == None:
        try:
            induce_prompt = induce + '[Text]:\n' + text
            text = chatGLM(induce_prompt,[])
            answer = extract_number(text)
        except Exception as e:
            print(e)
    # 如果仍旧抽不出来，则区第一个出现的数字
    if answer == None:    
        match = re.search(pattern=pattern_1,string=text)
        if match:
            answer = float(match.group())
            # print("Answer:", answer)
        else:
            answer =None
    return answer

In [ ]:
from tqdm import  tqdm

for i in tqdm(range(len(data))):
    ans = []
    answers = data[i]['answers']
    for item in answers:
        answer = extract_answer(item)
        ans.append(answer)
    data[i]['answers'] = ans
# data

In [ ]:
from utils import write_json

prompt_id_list = [201,202]

assert len(prompt_id_list) == 2
ids = prompt_id_list[0] +prompt_id_list[1]

res_p = f'{Result_Folder}/{dataset}/res-{format_time}-Type_{prompt_type}-{ids}-Num_{data_scale}-{path_num}.json'
# res_p = "./jx_result/gsm8k/res-1031_0933-Type_1-403-Num_100-5.json"

write_json(data,path=res_p)

## 筛选器

In [ ]:
import json

with open(res_p,'r',encoding='utf-8') as f:
    res_data = json.load(f)
len(res_data),res_data[0]

In [ ]:
from collections import Counter

for item in res_data:
    answers = item['answers']
    jx = Counter(answers).most_common(1)[0][0]
    item['pred_answer'] = jx

data[0]
    

In [ ]:
c1 = 0
c2 = 0
for item in res_data:
    pred = item['pred_answer']
    gold = item['answer']
    if pred:
        if float(pred) == float(gold):
            item['ans'] = True
            c1 += 1
        else:
            item['ans'] = False
            c2 += 1
    else:
        c2 += 1
assert c1 + c2 == len(data)
c1,c2

In [ ]:
res_p

In [ ]:
p = res_p.replace('res-','ses-')

write_json(res_data,p)

In [ ]:
1

## 测试数据集之间是否有重复

In [ ]:
import json

path = './raw_data/SVAMP/SVAMP.json'
path_1 = "./dataset/SVAMP/svamp_300_1.json"
p2 = "./dataset/SVAMP/svamp_400.json"
p3 = "./dataset/SVAMP/svamp_300_0.json"
data = []
data_1 = []
d2 = []
d3 = []
with open (path,'r') as f:
    data = json.load(f)
with open (path_1,'r') as f:
    data_1 = json.load(f)
with open (p2,'r') as f:
    d2 = json.load(f)    
with open (p3,'r') as f:
    d3 = json.load(f)        
len(data),len(data_1),len(d2),len(d3)

In [ ]:
data[0]

In [ ]:
q = [q['Question'] for q in data]
q1 = [q['Question'] for q in data_1]
q2 = [q['Question'] for q in d2]
q3 = [q['Question'] for q in d3]

In [ ]:
q == q3+q1+q2

In [ ]:
from collections import Counter

def find_duplicates(lst):
    counter = Counter(lst)
    duplicates = [element for element, count in counter.items() if count > 1]
    return duplicates

In [ ]:
a = find_duplicates(q)
a1 = find_duplicates(q1)
a2 = find_duplicates(q2)
len(a),len(a1),len(a2)

In [ ]:
s1 = set(q1)
s2 = set(q2)
s3 = set(q)
# t1 = [1,1,2]
t = set()
len(s1),len(s2),len(s3)

In [ ]:
# import json

# path = './jx_result/gsm8k/1018_1021-Type_1-303-Num_100-0.json'

# data = []
# with open(path,'r') as f:
#    for line in f:
#         data.append(line)
# len(data)

# count = 0
# pos = 0
# neg = 0
# for item in data:
#     if "\"ans\":" in item:
#         count+=1;
#         if "true" in item:
#             pos+=1
#         else:
#             neg+=1
# assert count == pos + neg,"统计错误"
# print(count,pos)
# print(pos/count)

In [ ]:
print('Brenda will have a total of $450 after 3 years. This can be calculated using the formula for simple interest: \n\nTotal Amount = Principal Amount + Interest Earned \n\nTotal Amount = 300 + (300 x 0.75 x 3) \n\nTotal Amount = 300 + 225 \n\nTotal Amount = $450')

In [ ]:
(203+199+198)/900

In [ ]:
(0.397+0.354+0.5)/3

In [ ]:
(184+33)/300

In [ ]:
199/300

In [ ]:
import json 
f1 = []
f2 = []
with open('./jx_result/SVAMP/1103_1520-Type_1-201-Num_400-5.json','r',encoding='utf-8') as f11:
    f1 = json.load(f11)
with open('./jx_result/SVAMP/1106_1348-Type_1-202-Num_400-5.json','r',encoding='utf-8') as f22:
    f2 = json.load(f22)    
len(f1),len(f2)


# In[5]:


t1 = []
t2 = []
for i in range(len(f1)):
    print(i)
    temp1 = {}
    temp2 = {}
    q1 = f1[i]['question']
    v1 = f1[i]['chain-of-thought']
    temp1[q1] = v1
    t1.append(temp1)
    q2 = f2[i]['question']
    v2 = f2[i]['chain-of-thought']
    temp2[q2] = v2
    t2.append(temp2)
all_paths = []
all_paths.append(t1)
all_paths.append(t2)